In [2]:
!pip install pandas
!pip install requests
!pip install pdfplumber
!pip install jsonpickle
!pip install boilerpy3

     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 5.6 MB 5.2 MB/s 
     |████████████████████████████████| 139 kB 69.0 MB/s 
     |████████████████████████████████| 1.9 MB 42.5 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.28-py3-none-any.whl size=32240 sha256=f5d0b3eb59ca542ec95864cb1400f55c3c6962d7bccfd9a2af8939905dee0e85
  Stored in directory: /root/.cache/pip/wheels/f2/b1/a0/c0a77b756d580f53b3806ae0e0b3ec945a8d05fca1d6e10cc1
Successfully built pdfplumber


In [7]:
import pandas as pd
import os

## Data download 

We fetch the data from the Modern Slavery Statement Registry page. For each of those statements, we use the 'StatementURL' column to fetch the original notebook.


In [3]:
base_statement_urls = ["https://downloads.modern-slavery-statement-registry.service.gov.uk/publicdownloads/StatementSummaries2020.csv",
                       "https://downloads.modern-slavery-statement-registry.service.gov.uk/publicdownloads/StatementSummaries2021.csv"]

In [4]:
# Import requests
import requests
import bs4

In [8]:
all_statements = [pd.read_csv(statement) for statement in base_statement_urls]

In [9]:
all_statements_df = pd.concat(all_statements)

In [10]:
all_statements_df.head()

,StatementYear,OrganisationName,Address,SectorType,CompanyNumber,LastUpdated,GroupSubmission,ParentName,StatementURL,EmailAddressNoURL,StatementSummaryURL,StatementStartDate,StatementEndDate,ApprovingPerson,DateApproved,StatementIncludesOrgStructure,NoOrgStructureReason,StatementIncludesPolicies,NoPoliciesReason,StatementIncludesRisksAssessment,NoRiskAssessmentReason,StatementIncludesDueDiligence,NoDueDiligenceReason,StatementIncludesTraining,NoTrainingReason,StatementIncludesGoals,NoGoalsReason,OrganisationSectors,OtherOrganisationSector,Turnover,YearsProducingStatements,Policies,OtherPolicies,NoPolicies,Training,OtherTraining,NoTraining,WorkingConditionsEngagement,NoEngagement,SocialAudits,NoSocialAudits,GrievanceMechanisms,NoGrievanceMechanisms,OtherMonitoring,NoRisks,Risk1,Risk1Area,Risk1SupplyChainTier,Risk1GroupAffected,Risk1OtherGroupAffected,Risk1Location,Risk1Mitigation,Risk2,Risk2Area,Risk2SupplyChainTier,Risk2GroupAffected,Risk2OtherGroupAffected,Risk2Location,Risk2Mitigation,Risk3,Risk3Area,Risk3SupplyChainTier,Risk3GroupAffected,Risk3OtherGroupAffected,Risk3Location,Risk3Mitigation,ILOIndicatorsInStatement,NoILOIndicatorsInStatement,ILOIndicatorsActions,ILOIndicatorsNoActions,DemonstrateProgress
0,2020,Tullow (EA) Holdings Limited,NaN,Private,NaN,14/05/2021,Yes,TULLOW OIL PLC,https://www.tullowoil.com/application/files/71...,NaN,https://modern-slavery-statement-registry.serv...,01/01/2020,31/12/2020,Dorothy Thompson (Chair),21/04/2021,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,No,NaN,"Mining, metals, chemicals and resources (inclu...",NaN,£100 million to £500 million,1 to 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,Tullow Ghana Limited,NaN,Private,NaN,14/05/2021,Yes,TULLOW OIL PLC,https://www.tullowoil.com/application/files/71...,NaN,https://modern-slavery-statement-registry.serv...,01/01/2020,31/12/2020,Dorothy Thompson (Chair),21/04/2021,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,No,NaN,"Mining, metals, chemicals and resources (inclu...",NaN,£100 million to £500 million,1 to 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,Tullow Namibia Limited,NaN,Private,NaN,14/05/2021,Yes,TULLOW OIL PLC,https://www.tullowoil.com/application/files/71...,NaN,https://modern-slavery-statement-registry.serv...,01/01/2020,31/12/2020,Dorothy Thompson (Chair),21/04/2021,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,No,NaN,"Mining, metals, chemicals and resources (inclu...",NaN,£100 million to £500 million,1 to 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,Tullow Uganda Limited,NaN,Private,NaN,14/05/2021,Yes,TULLOW OIL PLC,https://www.tullowoil.com/application/files/71...,NaN,https://modern-slavery-statement-registry.serv...,01/01/2020,31/12/2020,Dorothy Thompson (Chair),21/04/2021,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,No,NaN,"Mining, metals, chemicals and resources (inclu...",NaN,£100 million to £500 million,1 to 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,Tullow Uganda Operations Pty Limited,NaN,Private,NaN,14/05/2021,Yes,TULLOW OIL PLC,https://www.tullowoil.com/application/files/71...,NaN,https://modern-slavery-statement-registry.serv...,01/01/2020,31/12/2020,Dorothy Thompson (Chair),21/04/2021,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,Yes,NaN,No,NaN,"Mining, metals, chemicals and resources (inclu...",NaN,£100 million to £500 million,1 to 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_company = all_statements_df
df_company['StatementURL'].head(5)

0    https://www.tullowoil.com/application/files/71...
1    https://www.tullowoil.com/application/files/71...
2    https://www.tullowoil.com/application/files/71...
3    https://www.tullowoil.com/application/files/71...
4    https://www.tullowoil.com/application/files/71...
Name: StatementURL, dtype: object

You can iterrate each row in the dataframe and download the corresponding data from the provided url

In [12]:
import requests
import re
def download_file(url, local_file_location):
    if not os.path.isfile(local_file_location):
      r = requests.get(url, allow_redirects=True)
      with open(local_file_location, 'wb') as f:
            f.write(r.content)

Store all the files in a temporary directory

In [13]:
LOCAL_FILE_FOLDER = 'data/'

In [14]:
if not os.path.exists(LOCAL_FILE_FOLDER):
  os.mkdir(LOCAL_FILE_FOLDER)

In [15]:
working_df = df_company

In [16]:
working_df.shape

(10525, 71)

In [ ]:
%%time
i = 0
for _, row in working_df.iterrows():
    url = row['StatementURL']
    try:
      # use the last part of the url as the local file name
      file_path = url.split('/')[-1]
      file_name = ''.join([LOCAL_FILE_FOLDER, file_path])
      if i%20==0:
        print(f"Row {i}")
      if file_path:
        download_file(url, file_name)
      else:
        print(f"Empty url file extension: {url} at {i}, attempting to download as webpage")
        if(len(url.split("/"))>4):
          print("Attempting download of statement as HTML")
          filename = re.sub("/$",".html",file_name)
          download_file(url, filename)
        else:
          print("Skipping download, seems to be a home page.")
        
    except:
      print(f"Impossible to download statement at row {i}, statement url {url}")
    i+=1
    #print('downloaded', file_name)

## Extract data from PDF

The following code provides an example of extracting data from PDF files using a Python library called pdfplumber. However, you can choose any library as you prefer. 

You would need to do your own research if you wish to extract textual data from image file format such as png or jpg. However, the basic idea is to OCR the image file and then extract the text from the OCRed file.

Note: there are scanned PDF files that should be treated as image files. 

## Extract data from HTML

The HTML is parsed using BS4

In [ ]:
import os
import logging
import jsonpickle
import pathlib
import urllib.parse
from bs4 import BeautifulSoup
from bs4.element import Comment
from boilerpy3 import extractors

logger = logging.getLogger()
logger.setLevel(logging.INFO)


def accepted_tags(element):
    TAGS = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6',
            'section', 'a', 'a href', 'href', 'ul']
    
    if element.name in TAGS:
        return True

    elif element.parent.name in TAGS:
        return True
  
    elif isinstance(element, Comment):
        return False
    
    elif len(element)== 0:
        return False
    
    return False

def processing_extracted_text(input_slice):
    list_sentences = []
    list_tags = ['h2', 'h3', 'h4']
    for text_element in input_slice:
        if text_element.parent.name == 'p' and text_element.name == 'a':

                if "href" in str(text_element) and 'slavery' in str(text_element).lower():
                    if text_element.text+" "+text_element['href'] not in list_sentences:
                        if text_element.text+" "+text_element['href'].strip():
                            list_sentences.append(text_element.text+" "+text_element['href'])
                else:
                    continue
        elif text_element.name == 'a':
            if "href" in str(text_element) and 'slavery' in str(text_element).lower():
                if text_element.text+" "+text_element['href'] not in list_sentences:
                    if text_element.text+" "+text_element['href'].strip():
                        list_sentences.append(text_element.text+" "+text_element['href'])
        try:
            if text_element.name == 'p' and text_element['class'] == 'copyright':
                continue
        except (AttributeError, KeyError):
            pass

        if text_element.parent.name == 'a' and text_element.name == 'p':
            continue
        if text_element.parent.name == 'a' and text_element.name == 'span':
            continue
        if text_element.parent.name == 'nav' and text_element.name in list_tags:
            continue
        if "email" in text_element.text.lower():
            continue
        if "a href" not in str(text_element) and text_element.name != 'a':
    
            splitted_text_element = text_element.text.split()
            cleaned_element = ' '.join(splitted_text_element)
            if cleaned_element not in list_sentences:
                list_sentences.append(cleaned_element)
    return list_sentences


def filter_tags(body):
    soup  = BeautifulSoup(body, "html.parser")
    texts = soup.findAll(name = ['p','a', 'h1', 'h2', 'h3','h4','li', 'ul'])
    accepted_headers = [ 'h1', 'h2']
    extracted_data1 = list(filter(accepted_tags, texts))
    extracted_data = list(dict.fromkeys(filter(None, extracted_data1)))
    list_sentences = []

    for i, text_element in enumerate(extracted_data):
        
        if text_element.name in accepted_headers and 'slavery' in str(text_element).lower():
        
            list_sentences = processing_extracted_text(extracted_data[i:])
            break

        else:
            list_sentences = processing_extracted_text(extracted_data)
            
            
    return "\n".join(list_sentences)


def extract_filtered_data(html_file):
    list_invalid_pages = ["page was not found", "url was not found", "page can’t be found","page isn't real", 
    "page is not real", "page not found", "error 404", "couldn't find the page"]

    with open (html_file, encoding='utf8',errors='replace') as f:
        file_contents = f.read()

        file_contents_without_metadata = filter_tags(file_contents).split("\n")
        for sentence in file_contents_without_metadata:
            if len(sentence.split())<=2:
                file_contents_without_metadata.remove(sentence)

        final_content = "\n".join(file_contents_without_metadata)
        
        if any(element in final_content.lower() for element in list_invalid_pages):
            return "No Modern Slavery statement found"
                
        elif len(final_content)==0:
            return "Nothing to return"
        
        else:
            return final_content 

Now that all the files have been downloaded, perform text extraction according to the file extension

In [ ]:
from os import listdir
from os.path import isfile, join
import pdfplumber

df_data = pd.DataFrame()
for _, row in working_df.iterrows():
    url = row['StatementURL']
    if url and not pd.isna(url):
      file_name = join(LOCAL_FILE_FOLDER, url.split('/')[-1])
      if os.path.isfile(file_name):
          all_text = ''
          
          if file_name.endswith('pdf'):
              print(file_name)
              try:
                with pdfplumber.open(file_name) as pdf:
                    for pdf_page in pdf.pages:
                        single_page_text = pdf_page.extract_text()
                        if single_page_text:
                            all_text = all_text + '\n' + single_page_text
              except:
                  print(f"Impossible to parse PDF {file_name}")
                  all_text = "ERROR PARSING PDF"
          elif file_name.endswith('html') :
              all_text = extract_filtered_data(file_name)

          row['extracted_text'] = all_text
          df_data = df_data.append(row)
    else:
      row['extracted_text'] = ''

In [ ]:
df_data.shape

In [ ]:
df_data

In [ ]:
df_data.to_csv('data.csv', sep='|')